In [1]:
import pandas as pd # 데이터
import numpy as np # 기본적인 기능
import matplotlib as mpl
import matplotlib.pyplot as plt # 시각화
import matplotlib.font_manager as fm # 한글폰트 적용
import seaborn as sns # 시각화
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import LabelEncoder

font_path = 'Malgun Gothic'
fontprop = fm.FontProperties(fname=font_path,size=18)

In [2]:
from sklearn.model_selection import train_test_split # 훈련방식
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [3]:
import warnings
warnings.filterwarnings('ignore')
#오류 뜨는거 보기싫어서 삭제

In [4]:
card = pd.read_csv('train.csv')

In [5]:
card

,index,gender,car,reality,child_num,income_total,income_type,edu_type,family_type,house_type,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,work_phone,phone,email,occyp_type,family_size,begin_month,credit
0,0,F,N,N,0,202500.0,Commercial associate,Higher education,Married,Municipal apartment,-13899,-4709,1,0,0,0,NaN,2.0,-6.0,1.0
1,1,F,N,Y,1,247500.0,Commercial associate,Secondary / secondary special,Civil marriage,House / apartment,-11380,-1540,1,0,0,1,Laborers,3.0,-5.0,1.0
2,2,M,Y,Y,0,450000.0,Working,Higher education,Married,House / apartment,-19087,-4434,1,0,1,0,Managers,2.0,-22.0,2.0
3,3,F,N,Y,0,202500.0,Commercial associate,Secondary / secondary special,Married,House / apartment,-15088,-2092,1,0,1,0,Sales staff,2.0,-37.0,0.0
4,4,F,Y,Y,0,157500.0,State servant,Higher education,Married,House / apartment,-15037,-2105,1,0,0,0,Managers,2.0,-26.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26452,26452,F,N,N,2,225000.0,State servant,Secondary / secondary special,Married,House / apartment,-12079,-1984,1,0,0,0,Core staff,4.0,-2.0,1.0
26453,26453,F,N,Y,1,180000.0,Working,Higher education,Separated,House / apartment,-15291,-2475,1,0,0,0,NaN,2.0,-47.0,2.0
26454,26454,F,Y,N,0,292500.0,Working,Secondary / secondary special,Civil marriage,With parents,-10082,-2015,1,0,0,0,Core staff,2.0,-25.0,2.0
26455,26455,M,N,Y,0,171000.0,Working,Incomplete higher,Single / not married,House / apartment,-10145,-107,1,0,0,0,Laborers,1.0,-59.0,2.0


In [6]:
def income (x):
    income = ''
    if x <= 3000 : income= "1분위"
    elif x <= 4100 : income= "2분위"
    elif x <= 5270 : income= "3분위"
    elif x <= 5700 : income= "4분위"
    elif x <= 7600 : income= "5분위"
    elif x <= 8550 : income= "6분위"
    elif x <= 11700 : income= "7분위"
    elif x <= 17550 : income= "8분위"
    else : income = "9분위"
    return income

In [7]:
cardfull_df = card.fillna(value='unemployed')

In [8]:
cardhangul = cardfull_df.rename(columns={'gender' : '성별',   # 이해하기 쉽게 칼럼을 한글로 변경
                      'car' : '차',
                      'reality' : '집',
                      'child_num' : '자녀 수',
                      'income_total' : '연간소득',
                      'income_type' : '소득분류',
                      'edu_type':'교육수준',
                      'family_type':'결혼여부',
                      'house_type':'생활방식',
                      'DAYS_BIRTH':'출생일',
                      'DAYS_EMPLOYED':'취직일',
                      'FLAG_MOBIL':'폰',
                      'work_phone':'업무용 폰',
                      'phone':'전화',
                      'email':'이메일',
                      'occyp_type' : '직업' ,
                      'family_size' : '가족규모',
                      'begin_month' : '카드발급일',
                      'credit':'신용등급'                        
                           })

In [9]:
cardhangul.head(2)

,index,성별,차,집,자녀 수,연간소득,소득분류,교육수준,결혼여부,생활방식,출생일,취직일,폰,업무용 폰,전화,이메일,직업,가족규모,카드발급일,신용등급
0,0,F,N,N,0,202500.0,Commercial associate,Higher education,Married,Municipal apartment,-13899,-4709,1,0,0,0,unemployed,2.0,-6.0,1.0
1,1,F,N,Y,1,247500.0,Commercial associate,Secondary / secondary special,Civil marriage,House / apartment,-11380,-1540,1,0,0,1,Laborers,3.0,-5.0,1.0


In [10]:
cards = cardhangul.loc[:,'연간소득'] #연간소득 데이터만 따로 불러오기
income_card=cards*0.1
cards.value_counts(normalize=False)

135000.0    3164
157500.0    2233
180000.0    2225
112500.0    2178
225000.0    2170
            ... 
227250.0       1
91530.0        1
39600.0        1
432000.0       1
215100.0       1
Name: 연간소득, Length: 249, dtype: int64

In [11]:
income_list=[]
for i  in income_card:
    income_list.append(income(i))

cardhangul['소득분위']=income_list
cardhangul

,index,성별,차,집,자녀 수,연간소득,소득분류,교육수준,결혼여부,생활방식,...,취직일,폰,업무용 폰,전화,이메일,직업,가족규모,카드발급일,신용등급,소득분위
0,0,F,N,N,0,202500.0,Commercial associate,Higher education,Married,Municipal apartment,...,-4709,1,0,0,0,unemployed,2.0,-6.0,1.0,9분위
1,1,F,N,Y,1,247500.0,Commercial associate,Secondary / secondary special,Civil marriage,House / apartment,...,-1540,1,0,0,1,Laborers,3.0,-5.0,1.0,9분위
2,2,M,Y,Y,0,450000.0,Working,Higher education,Married,House / apartment,...,-4434,1,0,1,0,Managers,2.0,-22.0,2.0,9분위
3,3,F,N,Y,0,202500.0,Commercial associate,Secondary / secondary special,Married,House / apartment,...,-2092,1,0,1,0,Sales staff,2.0,-37.0,0.0,9분위
4,4,F,Y,Y,0,157500.0,State servant,Higher education,Married,House / apartment,...,-2105,1,0,0,0,Managers,2.0,-26.0,2.0,8분위
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26452,26452,F,N,N,2,225000.0,State servant,Secondary / secondary special,Married,House / apartment,...,-1984,1,0,0,0,Core staff,4.0,-2.0,1.0,9분위
26453,26453,F,N,Y,1,180000.0,Working,Higher education,Separated,House / apartment,...,-2475,1,0,0,0,unemployed,2.0,-47.0,2.0,9분위
26454,26454,F,Y,N,0,292500.0,Working,Secondary / secondary special,Civil marriage,With parents,...,-2015,1,0,0,0,Core staff,2.0,-25.0,2.0,9분위
26455,26455,M,N,Y,0,171000.0,Working,Incomplete higher,Single / not married,House / apartment,...,-107,1,0,0,0,Laborers,1.0,-59.0,2.0,8분위


In [12]:
#출생일 드랍
#인덱스 드랍
#취직일 드랍
#레이블 인코딩으로 전부 바꾸기

In [13]:
carddrop=cardhangul.drop(['출생일','index','취직일','카드발급일'],axis =1)
carddrop

,성별,차,집,자녀 수,연간소득,소득분류,교육수준,결혼여부,생활방식,폰,업무용 폰,전화,이메일,직업,가족규모,신용등급,소득분위
0,F,N,N,0,202500.0,Commercial associate,Higher education,Married,Municipal apartment,1,0,0,0,unemployed,2.0,1.0,9분위
1,F,N,Y,1,247500.0,Commercial associate,Secondary / secondary special,Civil marriage,House / apartment,1,0,0,1,Laborers,3.0,1.0,9분위
2,M,Y,Y,0,450000.0,Working,Higher education,Married,House / apartment,1,0,1,0,Managers,2.0,2.0,9분위
3,F,N,Y,0,202500.0,Commercial associate,Secondary / secondary special,Married,House / apartment,1,0,1,0,Sales staff,2.0,0.0,9분위
4,F,Y,Y,0,157500.0,State servant,Higher education,Married,House / apartment,1,0,0,0,Managers,2.0,2.0,8분위
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26452,F,N,N,2,225000.0,State servant,Secondary / secondary special,Married,House / apartment,1,0,0,0,Core staff,4.0,1.0,9분위
26453,F,N,Y,1,180000.0,Working,Higher education,Separated,House / apartment,1,0,0,0,unemployed,2.0,2.0,9분위
26454,F,Y,N,0,292500.0,Working,Secondary / secondary special,Civil marriage,With parents,1,0,0,0,Core staff,2.0,2.0,9분위
26455,M,N,Y,0,171000.0,Working,Incomplete higher,Single / not married,House / apartment,1,0,0,0,Laborers,1.0,2.0,8분위


In [14]:
pd.set_option('display.max_rows',None)

In [34]:
encoder = LabelEncoder()
carddrop['성별']=encoder.fit_transform(carddrop['성별'].values)
carddrop['차']=encoder.fit_transform(carddrop['차'].values)
carddrop['집']=encoder.fit_transform(carddrop['집'].values)
carddrop['결혼여부']=encoder.fit_transform(carddrop['결혼여부'].values)
carddrop['교육수준']=encoder.fit_transform(carddrop['교육수준'].values)
carddrop['생활방식']=encoder.fit_transform(carddrop['생활방식'].values)
carddrop['직업']=encoder.fit_transform(carddrop['직업'].values)
carddrop['소득분위']=encoder.fit_transform(carddrop['소득분위'].values)
carddrop['소득분류']=encoder.fit_transform(carddrop['소득분류'].values)
carddrop=carddrop.astype({'연간소득':'int','가족규모':'int','신용등급':'int'})
carddrop.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26457 entries, 0 to 26456
Data columns (total 17 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   성별      26457 non-null  int64
 1   차       26457 non-null  int64
 2   집       26457 non-null  int64
 3   자녀 수    26457 non-null  int64
 4   연간소득    26457 non-null  int32
 5   소득분류    26457 non-null  int64
 6   교육수준    26457 non-null  int64
 7   결혼여부    26457 non-null  int64
 8   생활방식    26457 non-null  int64
 9   폰       26457 non-null  int64
 10  업무용 폰   26457 non-null  int64
 11  전화      26457 non-null  int64
 12  이메일     26457 non-null  int64
 13  직업      26457 non-null  int64
 14  가족규모    26457 non-null  int32
 15  신용등급    26457 non-null  int32
 16  소득분위    26457 non-null  int64
dtypes: int32(3), int64(14)
memory usage: 3.1 MB


In [30]:
carddrop.loc[carddrop['소득분류']==1]

,성별,차,집,자녀 수,연간소득,소득분류,교육수준,결혼여부,생활방식,폰,업무용 폰,전화,이메일,직업,가족규모,신용등급,소득분위
14,0,0,1,0,157500.0,1,4,3,1,1,0,1,0,18,1.0,2.0,7
18,0,0,1,0,216000.0,1,4,1,1,1,0,0,0,18,2.0,2.0,8
21,0,0,0,0,157500.0,1,4,1,1,1,0,1,0,18,2.0,2.0,7
24,0,0,0,0,202500.0,1,4,3,1,1,0,1,0,18,1.0,2.0,8
46,0,0,0,0,135000.0,1,4,4,1,1,0,1,0,18,1.0,2.0,7
48,0,0,1,0,99000.0,1,4,1,1,1,0,0,0,18,2.0,2.0,6
52,0,0,1,0,54000.0,1,4,1,1,1,0,1,0,18,2.0,2.0,3
58,0,0,1,0,144000.0,1,4,1,1,1,0,0,0,18,2.0,2.0,7
59,0,0,1,0,360000.0,1,1,1,1,1,0,1,0,18,2.0,2.0,8
60,1,1,1,0,112500.0,1,4,2,1,1,0,0,0,18,1.0,1.0,6
